## Loading

In [1]:
import cv2
import numpy as np
import pandas as pd
from moviepy.editor import VideoFileClip
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, Reshape, Input
from tensorflow.keras.models import Model
from sklearn.decomposition import PCA

c:\Users\nosso\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Main

In [2]:
# Helper function to extract frames from video
def extract_frames(video_path, frame_rate=1):
    clip = VideoFileClip(video_path)
    fps = clip.fps
    frame_list = []
    timestamps = []

    for i, frame in enumerate(clip.iter_frames()):
        if i % int(fps / frame_rate) == 0:
            frame_list.append(frame)
            timestamps.append(i / fps)

    return frame_list, timestamps

In [3]:
def process_frames_to_dataframe(frames, timestamps):
    def build_autoencoder(input_shape=(224, 224, 3)):
        input_layer = Input(shape=input_shape)

        # Encoder
        x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_layer)
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
        x = Flatten()(x)
        latent_space = Dense(64, activation='relu', name='latent_features')(x)

        # Decoder
        x = Dense(224 * 224 * 3, activation='relu')(latent_space)
        decoded_output = Reshape((224, 224, 3))(x)

        # Models
        autoencoder = Model(inputs=input_layer, outputs=decoded_output)
        encoder = Model(inputs=input_layer, outputs=latent_space)

        autoencoder.compile(optimizer='adam', loss='mse')
        return autoencoder, encoder

    def manual_kmeans(data, n_clusters, max_iter=10, tol=1e-4):
        # Randomly initialize centroids using a small sample of the data
        np.random.seed(0)
        centroids = data[np.random.choice(data.shape[0], n_clusters, replace=False)]

        for iteration in range(max_iter):
            # Compute distances and assign clusters (vectorized)
            distances = np.linalg.norm(data[:, None, :] - centroids[None, :, :], axis=2)
            labels = np.argmin(distances, axis=1)

            # Compute new centroids
            new_centroids = np.array([
                data[labels == cluster].mean(axis=0) if (labels == cluster).any() else centroids[cluster]
                for cluster in range(n_clusters)
            ])

            # Check for convergence
            if np.all(np.linalg.norm(new_centroids - centroids, axis=1) < tol):
                break

            centroids = new_centroids

        return centroids, labels

    print("Resizing frames...")
    resized_frames = np.array([cv2.resize(frame, (224, 224)) for frame in frames])
    resized_frames = resized_frames / 255.0

    autoencoder, encoder = build_autoencoder()
    autoencoder.summary()

    frames = np.array(resized_frames)

    print("Training the autoencoder...")
    autoencoder.fit(frames, frames, epochs=10, batch_size=32, verbose=1)

    print("Extracting latent features...")
    latent_features = encoder.predict(frames)

    print("Calculating brightness...")
    brightness = np.array([
        frame.sum() / (frame.shape[0] * frame.shape[1] * frame.shape[2]) for frame in frames
    ])

    print("Clustering for dominant colors...")
    reshaped_frames = frames.reshape(-1, 3)  # Flatten frames to pixel-level colors

    # Sample a subset of pixels for clustering
    sample_indices = np.random.choice(reshaped_frames.shape[0], size=10000, replace=False)
    sampled_pixels = reshaped_frames[sample_indices]

    # Perform manual K-Means clustering on the sampled pixels
    dominant_colors, _ = manual_kmeans(sampled_pixels, n_clusters=3)

    # Map dominant colors back to frames
    color_features = []
    for frame_pixels in tqdm(frames, desc="Mapping dominant colors to frames"):
        frame_pixels = frame_pixels.reshape(-1, 3)
        distances = np.linalg.norm(frame_pixels[:, None, :] - dominant_colors[None, :, :], axis=2)
        labels = np.argmin(distances, axis=1)

        # Calculate average colors for the frame
        color_sums = np.zeros((3, 3))
        count = np.zeros(3)
        for label, pixel in zip(labels, frame_pixels):
            color_sums[label] += pixel
            count[label] += 1

        avg_colors = np.divide(color_sums, count[:, None], where=(count[:, None] != 0))
        color_features.append(avg_colors.mean(axis=0))  # Combine per-cluster colors

    print("Reducing dimensions of latent features...")
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(latent_features)

    print("Combining features into a DataFrame...")
    data = {
        "timestamp": timestamps,
        "brightness": brightness,
        "color1": [c[0] for c in color_features],
        "color2": [c[1] for c in color_features],
        "color3": [c[2] for c in color_features],
        "feature1": reduced_features[:, 0],
        "feature2": reduced_features[:, 1],
    }
    feature_df = pd.DataFrame(data)

    return feature_df


In [4]:
# Main function to process video and return a DataFrame
def analyze_video(video_path):
    print("Extracting frames from video...")
    frames, timestamps = extract_frames(video_path, frame_rate=1)

    print("Analyzing frames...")
    df = process_frames_to_dataframe(frames, timestamps)

    return df

## Run

In [5]:
# Upload and analyze your .mp4 video file
video_path = "Test.mp4"
df_results = analyze_video(video_path)

# Display the DataFrame
print(df_results)

Extracting frames from video...
Analyzing frames...
Resizing frames...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1605632)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_features (Dense)         │ (None, 64)             │   102,760,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 150528)         │     9,784,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 224, 224, 3)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,549,920 (429.34 MB)

 Trainable params: 112,549,920 (429.34 MB)

 Non-trainable params: 0 (0.00 B)

Training the autoencoder...
Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 23s 636ms/step - loss: 0.1492
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 23s 667ms/step - loss: 0.0975
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 23s 673ms/step - loss: 0.0863
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 22s 657ms/step - loss: 0.0716
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 23s 664ms/step - loss: 0.0612
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 25s 731ms/step - loss: 0.0572
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 23s 676ms/step - loss: 0.0560
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 23s 676ms/step - loss: 0.0539
Epoch 9/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 23s 681ms/step - loss: 0.0538
Epoch 10/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 23s 683ms/step - loss: 0.0534
Extracting latent features...
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 147ms/step
Calculating brightness...
Clustering for dominant colors...


Mapping dominant colors to frames: 100%|██████████| 1088/1088 [01:30<00:00, 11.96it/s]


Reducing dimensions of latent features...
Combining features into a DataFrame...
      timestamp  brightness    color1    color2    color3   feature1  \
0           0.0    0.255209  0.197018  0.156835  0.112140  -4.160371   
1           1.0    0.237458  0.450994  0.376861  0.291577  -9.755545   
2           2.0    0.236062  0.448760  0.359054  0.293935 -10.310310   
3           3.0    0.202793  0.493187  0.392524  0.315901  -9.517145   
4           4.0    0.240167  0.504492  0.383613  0.298241  -5.882386   
...         ...         ...       ...       ...       ...        ...   
1083     1083.0    0.195149  0.438187  0.346028  0.286445 -12.147798   
1084     1084.0    0.192127  0.448377  0.331727  0.280010 -12.665433   
1085     1085.0    0.208806  0.478030  0.368756  0.305702 -10.774250   
1086     1086.0    0.209984  0.467856  0.379547  0.310829 -10.651777   
1087     1087.0    0.209733  0.464000  0.381375  0.313372 -10.561563   

       feature2  
0      4.124103  
1    -10.165192  


In [7]:
from sklearn.preprocessing import StandardScaler

# List of features to scale
features_to_scale = ['brightness', 'color1', 'color2', 'color3', 'feature1', 'feature2']

# Standardize the features
scaler = StandardScaler()
df_results[features_to_scale] = scaler.fit_transform(df_results[features_to_scale])

In [8]:
from sklearn.preprocessing import PolynomialFeatures

# Create interaction terms
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
interaction_features = poly.fit_transform(df_results[features_to_scale])

# Convert to DataFrame
interaction_df = pd.DataFrame(interaction_features, columns=poly.get_feature_names_out(features_to_scale))

# Add to original DataFrame
df_results = pd.concat([df_results, interaction_df], axis=1)

In [9]:
print(df_results.corr())

                     timestamp  brightness    color1    color2    color3  \
timestamp             1.000000   -0.129458 -0.057525 -0.060351 -0.063547   
brightness           -0.129458    1.000000  0.150482  0.155723  0.161081   
color1               -0.057525    0.150482  1.000000  0.996569  0.975294   
color2               -0.060351    0.155723  0.996569  1.000000  0.990161   
color3               -0.063547    0.161081  0.975294  0.990161  1.000000   
feature1             -0.084031    0.940707  0.114663  0.118842  0.123047   
feature2             -0.051372   -0.142381 -0.033447 -0.034023 -0.034598   
brightness           -0.129458    1.000000  0.150482  0.155723  0.161081   
color1               -0.057525    0.150482  1.000000  0.996569  0.975294   
color2               -0.060351    0.155723  0.996569  1.000000  0.990161   
color3               -0.063547    0.161081  0.975294  0.990161  1.000000   
feature1             -0.084031    0.940707  0.114663  0.118842  0.123047   
feature2    

In [12]:
df_results.to_csv('C:/Users/nosso/OneDrive/Desktop/CNN.csv')